# PCA manual implementation

Here, we implement Principal Component Analysis (or dimensionality reduction) for feature selection.

## Initial functions

In [21]:
# Import the required libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [22]:
class PCA:
    
    def __init__(self, num_components):
        self.num_components = num_components
        self.components     = None
        self.mean           = None
        self.variance_share = None
    
    
    def fit(self, X):
        """
        Find principal components
        """
        
        # data centering
        self.mean = np.mean(X, axis = 0)
        X        -= self.mean
        
        # calculate eigenvalues & vectors
        cov_matrix      = np.cov(X.T)
        values, vectors = np.linalg.eig(cov_matrix)
        
        # sort eigenvalues & vectors 
        sort_idx = np.argsort(values)[::-1]
        values   = values[sort_idx]
        vectors  = vectors[:, sort_idx]
        
        # store principal components & variance
        self.components = vectors[:self.num_components]
        self.variance_share = np.sum(values[:self.num_components]) / np.sum(values)
    
    
    def transform(self, X):
        """
        Transform data
        """
        
        # data centering
        X -= self.mean
        
        # decomposition
        return np.dot(X, self.components.T)

## Dataset Operations

In [23]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv("cleaned_shifted_data.csv")
    
oe = OneHotEncoder(sparse=False)
encoded = oe.fit_transform(pd.DataFrame(df['Station']))

In [24]:
oe.get_feature_names()

array(['x0_IITG ', 'x0_LGBI Airport ', 'x0_Pan Bazaar ',
       'x0_Railway Colony '], dtype=object)

In [25]:

one_hot_df = pd.DataFrame(encoded, columns=oe.get_feature_names())
df = pd.concat([df, one_hot_df], axis=1)

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['year'] = df['Timestamp'].dt.year
df['month'] = df['Timestamp'].dt.month
df['dayofweek'] = df['Timestamp'].dt.day_of_week

drop_cols = [0,1,2,12,14,16]
drop_cols = df.columns[drop_cols]
df.drop(drop_cols,axis=1,inplace=True)

X = df.drop('AQI_calculated_shifted',axis = 1)
y = df['AQI_calculated_shifted']

In [26]:
print(X.shape, y.shape)

(174762, 17) (174762,)


## PCA Functions

In [27]:
# carry out PCA to reduce the number of features from 10 to 7 components
pca = PCA(num_components = 7)  # initialize PCA object
pca.fit(X) # fit PCA on old data
X_pca = pca.transform(X) # transform datasets
X_pca

array([[ -1.13953143,  37.03042904, -58.15373783, ...,  11.9386136 ,
        -50.07669542,  13.07018644],
       [ -0.93831225,  36.90182928, -58.03797556, ...,  12.23933133,
        -49.83941661,   9.99124985],
       [ -0.61468367,  36.52321966, -58.09325186, ...,  12.29396125,
        -50.31117511,  12.02783208],
       ...,
       [  8.37122392,  30.68552213,  -1.69996473, ..., -13.40586962,
        -42.72771882,  39.25524837],
       [  7.9843526 ,  32.26506977, -36.15169875, ...,   3.02848392,
        -46.17233121,  36.62568228],
       [ -8.65880869, -40.55730187, -46.25307474, ...,  -0.26595098,
        -49.52923117,  34.90827246]])

In [28]:
print(X_pca.shape)

(174762, 7)


## Explained Variance

In [29]:
# check explained variance
print(f"Explained variance: {pca.variance_share:.4f}")

Explained variance: 0.9967


## Apply Lasso, Ridge on PCA dataset

In [30]:
print(X_pca.shape, y.shape)

(174762, 7) (174762,)


In [31]:
# Splitting dataset into train and test set 
X_train, X_test, Y_train, Y_test = train_test_split( X_pca, y, test_size = 1 / 3, random_state = 0 )
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(116508, 7)
(58254, 7)
(116508,)
(58254,)


### Lasso

In [32]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
parameters = {'alpha': 0.5,
              'tol': 0.1
              }
model = Lasso(**parameters)

model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)
print("R2 {}".format(score))
mse = mean_squared_error(Y_test, Y_pred)
print("MSE", mse)

R2 0.3949083753550162
MSE 6767.704599349342


In [33]:
# Calculate the custom error metric
from error_define import *
Y_pred1 = Y_pred.reshape(-1, 1)
Y_test1 = Y_test.values.reshape(-1, 1)
downside_square_error(Y_pred1,Y_test1)

array([4325.58518664])

### Ridge

In [34]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
parameters = {'alpha': 0.5
              }
model = Ridge(**parameters)

model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)
print("R2 {}".format(score))
mse = mean_squared_error(Y_test, Y_pred)
print("MSE", mse)

R2 0.39498690735870423
MSE 6766.8262507143445


In [35]:
# Calculate the custom error metric
from error_define import *
Y_pred1 = Y_pred.reshape(-1, 1)
Y_test1 = Y_test.values.reshape(-1, 1)
downside_square_error(Y_pred1,Y_test1)

array([4320.00305746])